In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline




# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

In [2]:
import sklearn
sklearn.__version__

'0.23.1'

In [3]:
train_data_raw = pd.read_json('/Users/kanikamahajan/Desktop/MIDS/W207/Projects/Final_project_data/train.json')
test_data_raw = pd.read_json('/Users/kanikamahajan/Desktop/MIDS/W207/Projects/Final_project_data/test.json')
train_data_raw.head(4)


,giver_username_if_known,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,post_was_edited,request_id,request_number_of_comments_at_retrieval,request_text,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,...,requester_received_pizza,requester_subreddits_at_request,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,requester_user_flair,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc
0,N/A,0,1,0,t3_l25d7,0,Hi I am in need of food for my 4 children we a...,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,...,False,[],0,1,0,1,None,nickylvst,1317852607,1317849007
1,N/A,2,5,0,t3_rcb83,0,I spent the last money I had on gas today. Im ...,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,...,False,"[AskReddit, Eve, IAmA, MontereyBay, RandomKind...",34,4258,116,11168,None,fohacidal,1332652424,1332648824
2,N/A,0,3,0,t3_lpu5j,0,My girlfriend decided it would be a good idea ...,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,...,False,[],0,3,0,3,None,jacquibatman7,1319650094,1319646494
3,N/A,0,1,1,t3_mxvj3,4,"It's cold, I'n hungry, and to be completely ho...","It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,...,False,"[AskReddit, DJs, IAmA, Random_Acts_Of_Pizza]",54,59,76,81,None,4on_the_floor,1322855434,1322855434


In [4]:
# Generate result using pandas
day_request = []
for day in train_data_raw["unix_timestamp_of_request"]:
    date = time.ctime(int(day))
    day = date[0:3]
    day_request.append(day)

train_data_raw["day_of_request"] = day_request

## Final text processing
* Text pre-processing (removing punctuations, lower case etc.)
* NLTK stemming, lemetization
* TfidfVectorizer
* Latent Semantic Analysis and SVD for dimensionality reduction
* unix_timestamp_of_request_utc to day of week



In [ ]:
# from sklearn.preprocessing import StandardScaler
# x = train_data_raw.iloc[:, 6] #features
# x = StandardScaler().fit_transform(x) # normalizing the features

In [5]:
# Only lookign at text vector, using text as features and testing model accuracy
from sklearn.model_selection import train_test_split

# Extracting text from df (x) and outcome y

x = train_data_raw.iloc[:, 6] #features
y = train_data_raw.iloc[:, 22] #outcome

#test data does not have the outcome column so splitting training data into test, dev and training
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
train_data, test_data, train_labels, test_labels = train_test_split(x, y, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
dev_data, test_data, dev_labels, test_labels = train_test_split(test_data, test_labels, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1) 

#print(X_train, X_dev, X_test)
print(train_data.shape)


(3030,)


In [6]:
#New version
#Pre-processign text with TFIDF vectorizer and SVD dimesnionlaity reduction

def better_preprocessor(s):
    
    ### STUDENT START ###
    s = s.lower() # to lowercase
    
    #Stemming and lemmatization
    ps = PorterStemmer()
    lemmer = WordNetLemmatizer()
    words = word_tokenize(s)
    cleaned_words = [lemmer.lemmatize(ps.stem(word)) for word in words]
    
    #Remove stop words
    stop_words = set(stopwords.words('english'))
    new_cleaned_words = [w for w in cleaned_words if not w.lower() in stop_words]
    s = ''
    for word in new_cleaned_words:
        if word == '.' or word == '?' or word == ',':
            s = s + word
        else:
            s = s + ' ' + word
    
    return s
    ### STUDENT END ###


def P5():
    ### STUDENT START ###
    
    #Logistic Regression model (with no preprocessing of text)
    vectorizer = CountVectorizer(preprocessor= lambda x: x)
    X_train_counts_raw = vectorizer.fit_transform(train_data )
    print('Size of train Vocabulary with empty pre-processor is:', X_train_counts_raw.shape)
    
    #Logistic Regression model (with better pre-processor) and tfidfvectorizer
    
    #declare Vectorizer
    tfidf_vectorizer = TfidfVectorizer(preprocessor = better_preprocessor)

    #vectorizer on train data
    X_train_counts = tfidf_vectorizer.fit_transform(train_data)
    print('---------- \n Size of train Vocabulary with better preprocessor is:', X_train_counts.shape)
    
    #feature names
    feature_names = tfidf_vectorizer.get_feature_names()
    print(len(feature_names))
    
    #SVD for dimensionlaity reduction
    svd = TruncatedSVD(n_components=1500,n_iter=10,random_state=42)
    
    

    ## first reduced by SVD
    X_svd = svd.fit_transform(X_train_counts)
    print('---------- \n Size of train Vocabulary with better preprocessor and SVD is:', X_svd.shape)
    
    #explained variance by SVD reduced dimensions
    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))
    
    best_features = [feature_names[i] for i in svd.components_[0].argsort()[::-1]]
    #print(best_features[:1500])
    
    #print(type(X_train_counts.toarray()))
    
    #Convert to pandas df
    df_orig = pd.DataFrame(X_train_counts.toarray(), columns=feature_names)
    df = pd.DataFrame(df_orig, columns=best_features[:1500])
    #print(df)

    return (df)
    ### STUDENT END ###

P5()

# Link: https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
#SVD: https://scikit-learn.org/stable/modules/decomposition.html
#writeout featurenames: https://stackoverflow.com/questions/44633571/how-can-i-get-the-feature-names-from-sklearn-truncatedsvd-object/49055864

Size of train Vocabulary with empty pre-processor is: (3030, 12637)
---------- 
 Size of train Vocabulary with better preprocessor is: (3030, 8133)
8133
---------- 
 Size of train Vocabulary with better preprocessor and SVD is: (3030, 1500)
Explained variance of the SVD step: 90%


,pizza,would,thi,get,help,thank,realli,week,pay,food,...,mouth,toronto,honor,attack,variou,event,scholarship,film,36,americorp
0,0.000000,0.000000,0.000000,0.295420,0.000000,0.000000,0.000000,0.000000,0.000000,0.24076,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.037988,0.000000,0.000000,0.000000,0.000000,0.054353,0.062028,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.135567,0.118212,0.045682,0.046379,0.000000,0.048492,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.054036,0.188473,0.072834,0.073945,0.000000,0.000000,0.088232,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.112187,0.065217,0.075608,0.000000,0.000000,0.080258,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,0.037165,0.086419,0.000000,0.000000,0.109709,0.106350,0.060684,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3026,0.225919,0.065666,0.076128,0.000000,0.000000,0.080811,0.184447,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3028,0.000000,0.000000,0.064975,0.197900,0.142301,0.137943,0.000000,0.160915,0.244596,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
Text_features = P5()
Text_features.to_csv('/Users/kanikamahajan/Desktop/MIDS/W207/Projects/text_processed_features.csv')


Size of train Vocabulary with empty pre-processor is: (3030, 12637)
---------- 
 Size of train Vocabulary with better preprocessor is: (3030, 8133)
8133
---------- 
 Size of train Vocabulary with better preprocessor and SVD is: (3030, 1500)
Explained variance of the SVD step: 90%


In [15]:
#Combine new text processed features with main dataframe
#print(train_data.info())
df_text_time_merge = pd.concat([train_data_raw, Text_features], axis=1, join="inner")
df_text_time_merge.to_csv('/Users/kanikamahajan/Desktop/MIDS/W207/Projects/df_text_time_merge.csv')


# MULTIPLE CLASSIFIERS TO SEE ACCURACY WITH ONLY TEXT

In [ ]:
#vectorizer => transformer => classifier pipeline using pipeline class in sklearn
#Logistic Regression
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(penalty='l2', C=0.5,solver="liblinear", multi_class="auto")),])
text_clf.fit(train_data, train_labels)
predicted = text_clf.predict(dev_data)
np.mean(predicted == dev_labels)

In [ ]:
#SVM
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)),])
text_clf.fit(train_data, train_labels)
predicted = text_clf.predict(dev_data)
np.mean(predicted == dev_labels)


In [ ]:
#GRID Search for tuning hyperparameters

from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),
                  'clf__alpha': (1e-2, 1e-3),
                 }

gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(train_data, train_labels)
train_data[gs_clf.predict()]
gs_clf.best_score_
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

In [ ]:
#NB
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', BernoulliNB()),])
text_clf.fit(train_data, train_labels)
predicted = text_clf.predict(dev_data)
np.mean(predicted == dev_labels)



In [ ]:
#K nearest neighbors
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', KNeighborsClassifier(n_neighbors=5)),])
text_clf.fit(train_data, train_labels)
predicted = text_clf.predict(dev_data)
np.mean(predicted == dev_labels)


In [ ]:
# Multi layer Neural networks
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)),])
text_clf.fit(train_data, train_labels)
predicted = text_clf.predict(dev_data)
np.mean(predicted == dev_labels)

#Link: https://scikit-learn.org/stable/modules/neural_networks_supervised.html

In [ ]:
## OLD
from sklearn.feature_extraction.text import CountVectorizer

# Extracting post text from df
text_vector = train_data_raw.iloc[:, 6]

#Using Countvectorizer to convert into bag of words
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit_transform(text_vector)


# text into the vectorizer to get back counts
vector = vectorizer.transform(text_vector)

# Our final vector:
print('Full vector: ')
text_np = vector.toarray()

#Convert to pandas df
df = pd.DataFrame(text_np, columns=vectorizer.get_feature_names())
df.head()

df.info()

In [ ]:
#Selecting features and outcome data columns 

cols = df_new.columns
desired_cols = list([cols[2], cols[5], cols[10], cols[12], cols[16], cols[18], cols[20], cols[21], cols[25], cols[32]]) + list(cols[32:])
#df_new[desired_cols].head(100)

x = df_new[desired_cols] #features
y = df_new.iloc[:, 22]



In [ ]:
#test data does not have the outcome column so splitting training data into test, dev and training
from sklearn.model_selection import train_test_split
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1) 

#print(X_train, X_dev, X_test)
print(X_test.shape)
df_new.shape

In [ ]:
#test data does not have the outcome column so splitting training data into test and training
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42) 

In [ ]:
#K neighbors classifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_dev)

In [ ]:
print(confusion_matrix(y_dev, y_pred))
print(classification_report(y_dev, y_pred))
report = classification_report(y_dev, y_pred, output_dict=True)
print("Accuracy for 5 Nearest Neighbor model", report['accuracy'])

In [ ]:
#Logistic Regression

# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(max_iter= 1000)

# fit the model with data
logreg.fit(X_train,y_train)

#
y_pred=logreg.predict(X_dev)

In [ ]:
print(confusion_matrix(y_dev, y_pred))
print(classification_report(y_dev, y_pred))
report_log = classification_report(y_dev, y_pred, output_dict=True)
print("Accuracy for Logistic regression model is", report_log['accuracy'])

In [ ]:
# import the metrics class for printing confusion matrix
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_dev, y_pred)
cnf_matrix

In [ ]:
#Visualizing Confusion Matrix using Heatmap
import seaborn as sns
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
##SAMPLE TEXT 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
sample_text = ["One of the most basic ways we can numerically represent words", 
               "is through the one-hot encoding method (also sometimes called ",
               "count vectorizing)."]
vectorizer.fit(sample_text)
print('Vocabulary: ')
print(vectorizer.vocabulary_)

In [ ]:
# text into the vectorizer to get back counts
vector = vectorizer.transform(sample_text)

# Our final vector:
print('Full vector: ')
print(vector.toarray())
print(vector)

import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
new_corpus=[' '.join([stemmer.stem(word) for word in text.split(' ')])
          for text in sample_text]
print(new_corpus)